<a href="https://github.com/s-azaar/document_layout_segmentation" target="_parent"><img src="https://img.icons8.com/color/2x/github-2.png" alt="Open In GITHUB"/><p>Open using GitHub</p></a>


# Table of Contents



**1. Installation Instructions**

  **2.   Data Preprocessing**

**3. Model Building**

# Installation Instructions 


1.   **Clone** our GitHub repository


In [1]:
! git clone https://github.com/saleembilal-S/document_layout_segmentation.git

Cloning into 'document_layout_segmentation'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 63 (delta 9), reused 60 (delta 7), pack-reused 0
Unpacking objects: 100% (63/63), done.


2.   **Download** the requirements of yolov3



In [ ]:
! cd document_layout_segmentation/yolov3/installation && cat requirements_colab.txt | xargs -n 1 -L 1 pip install

https://drive.google.com/file/d/1i9POIJOcz3TpOmaeMcCukqWAFOcMMLmO/view?usp=sharing

3.  **Download** PRImA dataset


In [1]:
! wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1i9POIJOcz3TpOmaeMcCukqWAFOcMMLmO' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1i9POIJOcz3TpOmaeMcCukqWAFOcMMLmO" -O PRImA_Layout_Analysis_Dataset.zip && rm -rf /tmp/cookies.txt


--2020-11-30 22:02:47--  https://docs.google.com/uc?export=download&confirm=hmzZ&id=1i9POIJOcz3TpOmaeMcCukqWAFOcMMLmO
Resolving docs.google.com (docs.google.com)... 172.217.164.142, 2607:f8b0:4004:814::200e
Connecting to docs.google.com (docs.google.com)|172.217.164.142|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-24-docs.googleusercontent.com/docs/securesc/bmu5ugclaist950j5v0nm1ho4fupt3jl/9pskelbrsjrf0n7597e8b5h053fhd77b/1606773750000/04573370379016730842/14308130243757539325Z/1i9POIJOcz3TpOmaeMcCukqWAFOcMMLmO?e=download [following]
--2020-11-30 22:02:47--  https://doc-0s-24-docs.googleusercontent.com/docs/securesc/bmu5ugclaist950j5v0nm1ho4fupt3jl/9pskelbrsjrf0n7597e8b5h053fhd77b/1606773750000/04573370379016730842/14308130243757539325Z/1i9POIJOcz3TpOmaeMcCukqWAFOcMMLmO?e=download
Resolving doc-0s-24-docs.googleusercontent.com (doc-0s-24-docs.googleusercontent.com)... 172.217.7.225, 2607:f8b0:4004:802::2001
Connecting to doc-

# Data Preprocessing

1. **Unzip** the PRImA dataset.

In [4]:
! unzip -qq PRImA_Layout_Analysis_Dataset.zip


2.   **Convert** all of the images in the dataset from **TIFF** format to **JPEG** format because of their huge size  **-from 10GB to 1GB-** and also to reduce the training time.



In [ ]:
# Download the requirements of the conversion process
! pip install wand
! apt install imagemagick

In [6]:
# Import the necessary libraries 
import glob
from wand.image import Image
import os
from tqdm.notebook import tqdm

In [7]:
# Added files directory
dataset_dir = "PRImA Layout Analysis Dataset/";
ttif_dir_image = "Images/";
jpg_dir_image = "Images/"
jpg_dir="PRImA Layout Analysis Dataset_JPG_v1/";

In [8]:
# Run conversion process

if not os.path.exists(jpg_dir):
    os.makedirs(jpg_dir)

if not os.path.exists(jpg_dir+jpg_dir_image):
    os.makedirs(jpg_dir+jpg_dir_image)

for name in tqdm(glob.glob(dataset_dir+ttif_dir_image+'*.tif')):
    with Image(filename=name) as img:
        img.format = 'jpeg'
        name = name.split(".")
        name = name[0].split("/")
        img.save(filename=jpg_dir+jpg_dir_image+name[2]+'.jpg')


/usr/local/lib/python3.6/dist-packages/wand/image.py:9094: CoderWarning: Unknown field with tag 59932 (0xea1c) encountered. `TIFFReadDirectory' @ warning/tiff.c/TIFFWarnings/912
  self.raise_exception()


3. **Convert** the dataset formt from **VOC Pascal** formt to **YOLO** format

In [9]:
# Import the necessery libraries
import os
import sys
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import glob
import matplotlib.pyplot as plt
import xmltodict
import json
from tqdm.notebook import tqdm
from pycocotools.coco import COCO

In [10]:
# Added files directory
dataset_dir = "PRImA Layout Analysis Dataset/";
xml_dir = "XML/";
jpg_dir="PRImA Layout Analysis Dataset_JPG_v1/";


In [11]:

combined = [];
files = os.listdir(dataset_dir + xml_dir);

#label generation for csv
for i in tqdm(range(len(files))):
    annoFile = dataset_dir + xml_dir + files[i];
    f = open(annoFile, 'r');
    my_xml = f.read();

    anno= dict(dict(dict(xmltodict.parse(my_xml))['PcGts'])['Page'])
    
    wordd= "00001178"
    fnamee=""
    fname=""
    fname=files[i].split(".")[0]
    word = "pc-"                        
    if word in fname: 
      fname =fname.replace(word,"")
    fnamee=fname
    fname+=".jpg"

    width=int(anno['@imageWidth'])
    height=int(anno['@imageHeight'])

    label_str = ""

    for key in anno.keys():
        if(key=='@imageFilename' or key=='@imageWidth' or key=='@imageHeight'):
            continue
        if(key=="TextRegion"):
            if(type(anno["TextRegion"]) == list):
                for j in range(len(anno["TextRegion"])):
                    text=anno["TextRegion"][j]
                    xmin=width
                    ymin=height
                    xmax=0
                    ymax=0
                    if(text["Coords"]):
                        if(text["Coords"]["Point"]):
                            for k in range(len(text["Coords"]["Point"])):
                                coordinates=anno["TextRegion"][j]["Coords"]["Point"][k]
                                xmin= min(xmin, int(coordinates['@x']));
                                ymin= min(ymin, int(coordinates['@y']));
                                xmax= min(max(xmax, int(coordinates['@x'])), width);
                                ymax= min(max(ymax, int(coordinates['@y'])), height);
                            if('@type' in text.keys()):    
                                label_str+= str(xmin)+' '+str(ymin)+' '+str(xmax)+' '+str(ymax)+' '+text['@type']+' '
                              
            else:             
                text=anno["TextRegion"]
                xmin=width
                ymin=height
                xmax=0
                ymax=0
                if(text["Coords"]):
                    if(text["Coords"]["Point"]):
                        for k in range(len(text["Coords"]["Point"])):
                            coordinates=anno["TextRegion"]["Coords"]["Point"][k]
                            xmin= min(xmin, int(coordinates['@x']));
                            ymin= min(ymin, int(coordinates['@y']));
                            xmax= min(max(xmax, int(coordinates['@x'])), width);
                            ymax= min(max(ymax, int(coordinates['@y'])), height);
                        if('@type' in text.keys()):    
                            label_str+= str(xmin)+' '+str(ymin)+' '+str(xmax)+' '+str(ymax)+' '+text['@type']+' '
                            
        
        else:
            val=""
            if(key=='GraphicRegion'):
                val="graphics"
            elif(key=='ImageRegion'):
                val="image"
            elif(key=='NoiseRegion'):
                val="noise"
            elif(key=='ChartRegion'):
                val="chart"
            elif(key=='TableRegion'):
                val="table"
            elif(key=='SeparatorRegion'):
                val="separator"
            elif(key=='MathsRegion'):
                val="maths"
            elif(key=='LineDrawingRegion'):
                val="linedrawing"
            else:
                val="frame"

           
            if(type(anno[key]) == list):  
                #print(fname,anno[key])   
                if(fnamee == wordd):
                    print(anno[key]) 
                for j in range(len(anno[key])):
                    text=anno[key][j]
                    xmin=width
                    ymin=height
                    xmax=0
                    ymax=0
                    if(text["Coords"]):
                        if(text["Coords"]["Point"]):
                            for k in range(len(text["Coords"]["Point"])):
                                coordinates=anno[key][j]["Coords"]["Point"][k]
                                xmin= min(xmin, int(coordinates['@x']));
                                ymin= min(ymin, int(coordinates['@y']));
                                xmax= min(max(xmax, int(coordinates['@x'])), width);
                                ymax= min(max(ymax, int(coordinates['@y'])), height);
                        label_str+= str(xmin)+' '+str(ymin)+' '+str(xmax)+' '+str(ymax)+' '+ val +' '
                        
            else:
                text=anno[key]
                xmin=width
                ymin=height
                xmax=0
                ymax=0
                if(text["Coords"]):
                    if(text["Coords"]["Point"]):
                        for k in range(len(text["Coords"]["Point"])):
                            coordinates=anno[key]["Coords"]["Point"][k]
                            xmin= min(xmin, int(coordinates['@x']));
                            ymin= min(ymin, int(coordinates['@y']));
                            xmax= min(max(xmax, int(coordinates['@x'])), width);
                            ymax= min(max(ymax, int(coordinates['@y'])), height);  
                        label_str+= str(xmin)+' '+str(ymin)+' '+str(xmax)+' '+str(ymax)+' '+val+' '
                        
    
    label_str=label_str[:-1]
    combined.append([fname, label_str])

        

[OrderedDict([('@id', 'r3'), ('@orientation', '0.00000'), ('@colour', 'black'), ('Coords', OrderedDict([('Point', [OrderedDict([('@x', '107'), ('@y', '473')]), OrderedDict([('@x', '598'), ('@y', '473')]), OrderedDict([('@x', '564'), ('@y', '473')]), OrderedDict([('@x', '564'), ('@y', '474')]), OrderedDict([('@x', '225'), ('@y', '474')]), OrderedDict([('@x', '225'), ('@y', '473')])])]))]), OrderedDict([('@id', 'r21'), ('@orientation', '0.00000'), ('@colour', 'black'), ('Coords', OrderedDict([('Point', [OrderedDict([('@x', '2253'), ('@y', '0')]), OrderedDict([('@x', '2253'), ('@y', '49')]), OrderedDict([('@x', '2252'), ('@y', '49')]), OrderedDict([('@x', '2252'), ('@y', '98')]), OrderedDict([('@x', '2252'), ('@y', '97')]), OrderedDict([('@x', '1177'), ('@y', '97')]), OrderedDict([('@x', '1177'), ('@y', '96')]), OrderedDict([('@x', '102'), ('@y', '96')]), OrderedDict([('@x', '102'), ('@y', '102')]), OrderedDict([('@x', '2254'), ('@y', '102')]), OrderedDict([('@x', '2253'), ('@y', '102')])

4. **Save** the lables in a **CSV** file

In [12]:
df = pd.DataFrame(combined, columns = ['ID', 'Label']);
df.to_csv(jpg_dir + "/train_labels.csv", index=False);

5. **Convert** the CSV file to **YOLO** files

In [13]:
import pandas as pd


jpg_dir="PRImA Layout Analysis Dataset_JPG_v1/";
jpg_dir_image = "Images/";
anno_file = "PRImA Layout Analysis Dataset_JPG_v1/train_labels.csv";
labels_dir = "labels/";
classes_file = "classes.txt";


labels_dir_all = jpg_dir + labels_dir
if(not os.path.isdir(labels_dir_all)):
    os.mkdir(labels_dir_all);



df = pd.read_csv( anno_file);
len(df)


columns = df.columns
classes = [];
for i in range(len(df)):
    img_file = df[columns[0]][i];
    labels = df[columns[1]][i];
    tmp = labels.split(" ");
    for j in range(len(tmp)//5):
        label = tmp[j*5 + 4];
        if(label not in classes):
            classes.append(label);
classes = sorted(classes)
classes

f = open(jpg_dir + classes_file, 'w');
for i in range(len(classes)):
    f.write(classes[i]);
    f.write("\n");
f.close();


for i in tqdm(range(len(df))):
    img_file = df[columns[0]][i];
    labels = df[columns[1]][i];
    tmp = labels.split(" ");
    fname = labels_dir_all  + img_file.split(".")[0] + ".txt";
    img = Image.open(jpg_dir  + jpg_dir_image  + img_file);
    width, height = img.size
    
    f = open(fname, 'w');
    for j in range(len(tmp)//5):
        x1 = float(tmp[j*5 + 0]);
        y1 = float(tmp[j*5 + 1]);
        x2 = float(tmp[j*5 + 2]);
        y2 = float(tmp[j*5 + 3]);
        label = tmp[j*5 + 4];
        
        x_c = str(((x1 + x2)/2)/width);
        y_c = str(((y1 + y2)/2)/height);
        w = str((x2 - x1)/width);
        h = str((y2 - y1)/height);
        index = str(classes.index(label));
        
        f.write(index + " " + x_c + " " + y_c + " " + w + " " + h);
        f.write("\n");
    f.close();

# Model Building 

1. **Import** all necessary classes

In [14]:
import os
import sys
sys.path.append("document_layout_segmentation/yolov3/lib")
from train_detector import Detector
detect_image = Detector();

2. **Load** all necessary directory

In [15]:
#dataset directories
img_dir = "PRImA Layout Analysis Dataset_JPG_v1/Images/"
label_dir = "PRImA Layout Analysis Dataset_JPG_v1/labels/"
class_list_file = "PRImA Layout Analysis Dataset_JPG_v1/classes.txt"

3. **Start** learning process

In [ ]:
detect_image.set_model(model_name="yolov3");
detect_image.set_train_dataset(img_dir, label_dir, class_list_file, batch_size=16)
detect_image.set_val_dataset(img_dir, label_dir)
detect_image.set_hyperparams(optimizer="sgd", lr=0.003, multi_scale=False, evolve=False);
detect_image.Train(num_epochs=30);

Using CUDA Apex device0 _CudaDeviceProperties(name='Tesla T4', total_memory=15079MB)




Caching labels (478 found, 0 missing, 0 empty, 1 duplicate, for 478 images): 100%|██████████| 478/478 [00:00<00:00, 5363.55it/s]

Caching labels (478 found, 0 missing, 0 empty, 1 duplicate, for 478 images): 100%|██████████| 478/478 [00:00<00:00, 4867.96it/s]


Model Summary: 222 layers, 6.16153e+07 parameters, 6.16153e+07 gradients
Using 2 dataloader workers
Starting training for 30 epochs...

     Epoch   gpu_mem      GIoU       obj       cls     total   targets  img_size



  0%|          | 0/30 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/torch/cuda/memory.py:346: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  FutureWarning)

      0/29      4.8G      8.07      17.6        32      57.6       312       416:   0%|          | 0/30 [00:21<?, ?it/s]
      0/29      4.8G      8.07      17.6        32      57.6       312       416:   3%|▎         | 1/30 [00:21<10:25, 21.56s/it]
      0/29     4.81G      8.06      17.5      31.8      57.4       309       416:   3%|▎         | 1/30 [00:22<10:25, 21.56s/it]
      0/29     4.81G      8.06      17.5      31.8      57.4       309       416:   7%|▋         | 2/30 [00:22<07:07, 15.26s/it]
      0/29     4.82G      8.12      19.2      31.8      59.1       422       416:   7%|▋         | 2/30 [00:30<07:07, 15.26s/it]
      0/29     4.82G      8.12      19.2      31.8      59.1       422       416:  10%|█         | 3/30 [00:30<05:58, 13.29s/it]
      0/29     4.82G     

                 all       478  1.04e+04         0         0         0         0

     Epoch   gpu_mem      GIoU       obj       cls     total   targets  img_size



  0%|          | 0/30 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/torch/cuda/memory.py:346: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  FutureWarning)

      1/29     6.15G      8.02      21.6      31.8      61.5       395       416:   0%|          | 0/30 [00:13<?, ?it/s]
      1/29     6.15G      8.02      21.6      31.8      61.5       395       416:   3%|▎         | 1/30 [00:13<06:38, 13.75s/it]
      1/29     6.15G      8.14        21      31.7      60.8       376       416:   3%|▎         | 1/30 [00:14<06:38, 13.75s/it]
      1/29     6.15G      8.14        21      31.7      60.8       376       416:   7%|▋         | 2/30 [00:14<04:33,  9.76s/it]
      1/29     6.15G      8.09      20.6      31.7      60.4       358       416:   7%|▋         | 2/30 [00:27<04:33,  9.76s/it]
      1/29     6.15G      8.09      20.6      31.7      60.4       358       416:  10%|█         | 3/30 [00:27<04:49, 10.71s/it]
      1/29     6.16G     